In [112]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import test_func as tf
%matplotlib inline

In [113]:
files = glob.glob("train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)

In [114]:
df = tf.data_pre(df)
model = tf.model_lgb(df)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.010389 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11686
[LightGBM] [Info] Number of data points in the train set: 509880, number of used features: 17
[LightGBM] [Info] Start training from score 7.217366
[1]	valid_0's l1: 0.252214
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.235956
[3]	valid_0's l1: 0.221732
[4]	valid_0's l1: 0.209005
[5]	valid_0's l1: 0.19773
[6]	valid_0's l1: 0.188016
[7]	valid_0's l1: 0.17909
[8]	valid_0's l1: 0.171536
[9]	valid_0's l1: 0.164668
[10]	valid_0's l1: 0.158413
[11]	valid_0's l1: 0.15288
[12]	valid_0's l1: 0.147911
[13]	valid_0's l1: 0.143688
[14]	valid_0's l1: 0.139724
[15]	valid_0's l1: 0.136032
[16]	valid_0's l1: 0.132881
[17]	valid_0's l1: 0.129718
[18]	valid_0's l1: 0.127098
[19]	valid_0's l1: 0.124808
[20]	va

In [58]:
def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
    df = df.drop(nonnull_list, axis=1)

    df = df.drop("市区町村名", axis=1)

    df = df.drop("種類", axis=1)

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)


    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    year = {
        "年第１四半期": ".25",
        "年第２四半期": ".50",
        "年第３四半期": ".75",
        "年第４四半期": ".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)
    
    for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
                df[col] = df[col].astype("category")
    
    return df

In [59]:
df = data_pre(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637351 entries, 40108844 to 12009989
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       637351 non-null  int64   
 1   都道府県名         637351 non-null  category
 2   地区名           637060 non-null  category
 3   最寄駅：名称        634732 non-null  category
 4   最寄駅：距離（分）     614306 non-null  float64 
 5   間取り           615609 non-null  category
 6   面積（㎡）         637351 non-null  float64 
 7   建築年           619117 non-null  float64 
 8   建物の構造         623423 non-null  category
 9   用途            591214 non-null  category
 10  今後の利用目的       275091 non-null  category
 11  都市計画          618786 non-null  category
 12  建ぺい率（％）       614848 non-null  float64 
 13  容積率（％）        614848 non-null  float64 
 14  取引時点          637351 non-null  float64 
 15  改装            580431 non-null  category
 16  取引の事情等        18275 non-null   category
 17  取引価格（総額）_log  637351

In [96]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

df_train, df_val =train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)


In [61]:
vals = model.predict(val_x)
mae(vals, val_y)

0.07680858738862091

In [69]:
df_test = pd.read_csv("test.csv", index_col=0)
df_test = data_pre(df_test)

In [70]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("submit_test.csv")

In [68]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 637351 entries, 40108844 to 12009989
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       637351 non-null  int64   
 1   都道府県名         637351 non-null  category
 2   地区名           637060 non-null  category
 3   最寄駅：名称        634732 non-null  category
 4   最寄駅：距離（分）     614306 non-null  float64 
 5   間取り           615609 non-null  category
 6   面積（㎡）         637351 non-null  float64 
 7   建築年           619117 non-null  float64 
 8   建物の構造         623423 non-null  category
 9   用途            591214 non-null  category
 10  今後の利用目的       275091 non-null  category
 11  都市計画          618786 non-null  category
 12  建ぺい率（％）       614848 non-null  float64 
 13  容積率（％）        614848 non-null  float64 
 14  取引時点          637351 non-null  float64 
 15  改装            580431 non-null  category
 16  取引の事情等        18275 non-null   category
 17  取引価格（総額）_log  637351

In [89]:
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
地区名,8476
最寄駅：名称,6352
建築年,3527
取引時点,3183
面積（㎡）,2135
最寄駅：距離（分）,1854
間取り,932
市区町村コード,806
都道府県名,578
改装,503
